# Czyszczenie i jakość danych - Demo

**Cel szkoleniowy:** Opanowanie technik identyfikacji i rozwiązywania problemów jakości danych, zrozumienie strategii obsługi wartości pustych, walidacji typów, deduplikacji i standaryzacji danych

**Zakres tematyczny:**
- Obsługa wartości pustych: fillna(), dropna(), coalesce()
- Walidacja typów: cast(), to_date(), to_timestamp()
- Deduplikacja: dropDuplicates() - all columns vs key columns
- Standardyzacja: formaty dat, tekstów, kategorii
- Typowe problemy jakości: whitespace, niepoprawne kody, inconsistent formatting

## Kontekst i wymagania

- **Dzień szkolenia**: Dzień 1 - Fundamentals & Exploration
- **Typ notebooka**: Demo
- **Wymagania techniczne**:
  - Databricks Runtime 13.0+ (zalecane: 14.3 LTS)
  - Unity Catalog włączony
  - Uprawnienia: CREATE TABLE, CREATE SCHEMA, SELECT, MODIFY
  - Klaster: Standard z 2-4 workers
- **Czas trwania**: 20 minut
- **Prerekvizity**: 03_basic_transformations_sql_pyspark.ipynb

## Wstęp teoretyczny

**Cel sekcji:** Zrozumienie fundamentów data quality i technik czyszczenia danych

**Podstawowe pojęcia:**
- **Data Quality**: Miara przydatności danych do ich zamierzonego celu
- **Data Cleansing**: Proces identyfikacji i korygowania błędów w danych
- **Data Validation**: Weryfikacja zgodności danych z regułami biznesowymi
- **Data Standardization**: Ujednolicenie formatów i reprezentacji danych
- **Data Profiling**: Analiza struktury, zawartości i relacji w danych

**Wymiary jakości danych:**
- **Completeness**: Czy wszystkie wymagane dane są obecne
- **Accuracy**: Czy dane są poprawne i zgodne z rzeczywistością
- **Consistency**: Czy dane są spójne w całym systemie
- **Timeliness**: Czy dane są aktualne
- **Validity**: Czy dane spełniają reguły biznesowe
- **Uniqueness**: Czy rekordy są unikalne

**Dlaczego to ważne?**
Niska jakość danych prowadzi do błędnych analiz, złych decyzji biznesowych i utraty zaufania do systemu. Czyszczenie danych to często 60-80% czasu w projektach data engineering. Systematyczne podejście do data quality zapewnia wiarygodność całego pipeline'u danych.

## Izolacja per użytkownik

Uruchom skrypt inicjalizacyjny dla per-user izolacji katalogów i schematów:

In [0]:
%run ../00_setup

## Konfiguracja

Import bibliotek i ustawienie zmiennych środowiskowych:

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
import re
from datetime import datetime, timedelta

# Wyświetl kontekst użytkownika (zmienne z 00_setup)
print("=== Kontekst użytkownika ===")
print(f"Katalog: {CATALOG}")
print(f"Schema Bronze: {BRONZE_SCHEMA}")
print(f"Schema Silver: {SILVER_SCHEMA}")
print(f"Schema Gold: {GOLD_SCHEMA}")
print(f"Użytkownik: {raw_user}")

# Ustaw katalog i schemat jako domyślne
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {BRONZE_SCHEMA}")

print("\n=== Konfiguracja zakończona pomyślnie ===")

## Sekcja 1: Wczytanie danych z dataset

**Wprowadzenie teoretyczne:**

W tym notebooku wykorzystujemy pliki z lokalnego folderu `dataset/` (Dzień 1-2 szkolenia). Pliki CSV, JSON i Parquet są ładowane bezpośrednio z systemu plików używając ścieżki `DATASET_BASE_PATH` z `00_setup.ipynb`.

**Kluczowe pojęcia:**
- **DATASET_BASE_PATH**: Ścieżka do folderu dataset/ zdefiniowana w 00_setup.ipynb
- **CSV Reader**: spark.read.format("csv") z opcjami (header, inferSchema)
- **Schema inference**: Automatyczne wykrywanie typów vs jawna definicja schematu
- **File paths**: Absolutne ścieżki do plików w lokalnym systemie

**Zastosowanie praktyczne:**
- Wczytywanie raw files z folderu dataset/
- Eksploracja i profilowanie danych przed czyszczeniem
- Przygotowanie do załadowania do Delta tables (Bronze layer)

### Przykład 1.1: Wczytanie danych customers z dataset

**Cel:** Załadowanie danych klientów z pliku CSV przechowywanego w folderze dataset/

**Podejście:**
1. Zdefiniowanie ścieżki używając DATASET_BASE_PATH z 00_setup.ipynb
2. Wczytanie CSV z opcjami (header, inferSchema)
3. Podstawowa eksploracja załadowanych danych

In [0]:
# RESOURCE: CSV file: {DATASET_BASE_PATH}/customers/customers.csv
# VARIABLE: df_customers - DataFrame z danymi klientów

# Ścieżka do pliku w dataset
customers_path = f"{DATASET_BASE_PATH}/customers/customers.csv"

# Wczytanie danych
df_customers = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(customers_path)

# Podstawowa eksploracja
print("=== Dane klientów załadowane ===")
print(f"Liczba rekordów: {df_customers.count()}")
print(f"Liczba kolumn: {len(df_customers.columns)}")
print(f"\nKolumny: {df_customers.columns}")

# Schemat danych
print("\n=== Schemat danych ===")
df_customers.printSchema()

# Podgląd pierwszych rekordów
print("\n=== Pierwsze 10 rekordów ===")
display(df_customers.limit(10))

**Wyjaśnienie:**

Ścieżka do pliku używa zmiennej `DATASET_BASE_PATH` zdefiniowanej w `00_setup.ipynb`, która wskazuje na folder `dataset/` w workspace. Opcja `inferSchema=true` automatycznie wykrywa typy danych, co jest wygodne dla eksploracji, ale w produkcji zaleca się jawne definiowanie schematów dla wydajności i kontroli.

### Przykład 1.2: Data Profiling - analiza jakości danych

**Cel:** Identyfikacja problemów jakości w załadowanych danych przed rozpoczęciem czyszczenia

In [0]:
# RESOURCE: DataFrame df_customers
# VARIABLE: profiling_report - dict ze statystykami jakości

print("=" * 80)
print("DATA QUALITY PROFILING REPORT")
print("=" * 80)

# 1. Completeness - Analiza wartości null
print("\n1. COMPLETENESS - Wartości null per kolumna:")
print("-" * 80)

for col_name in df_customers.columns:
    null_count = df_customers.filter(F.col(col_name).isNull()).count()
    total_count = df_customers.count()
    null_pct = (null_count / total_count) * 100
    print(f"  {col_name:20s}: {null_count:4d} nulls ({null_pct:5.1f}%)")

# 2. Uniqueness - Analiza duplikatów
print("\n2. UNIQUENESS - Analiza duplikatów:")
print("-" * 80)
total_rows = df_customers.count()
unique_rows = df_customers.distinct().count()
duplicate_rows = total_rows - unique_rows
print(f"  Total rows: {total_rows}")
print(f"  Unique rows: {unique_rows}")
print(f"  Duplicate rows: {duplicate_rows}")
print(f"  Duplication rate: {(duplicate_rows/total_rows)*100:.1f}%")

# 3. Consistency - Analiza wartości unikalnych w kluczowych kolumnach
print("\n3. CONSISTENCY - Wartości unikalne:")
print("-" * 80)

for col_name in df_customers.columns:
    distinct_count = df_customers.select(col_name).distinct().count()
    print(f"  {col_name:20s}: {distinct_count:4d} unikalnych wartości")

# 4. Accuracy - Przykładowe wartości
print("\n4. ACCURACY - Przykładowe wartości (pierwsze 5):")
print("-" * 80)
display(df_customers.limit(5))

print("\n" + "=" * 80)
print("PROFILING COMPLETED")
print("=" * 80)

## Sekcja 2: Obsługa wartości pustych

**Wprowadzenie teoretyczne:**

Brakujące wartości są jednym z najczęstszych problemów jakości danych. Strategia obsługi null values zależy od kontekstu biznesowego i charakteru danych. Niepoprawna obsługa może prowadzić do błędów w analizach i modelach ML.

**Kluczowe pojęcia:**
- **fillna()**: Wypełnienie wartości null określoną wartością lub strategią
- **dropna()**: Usunięcie rekordów zawierających wartości null
- **coalesce()**: Wybór pierwszej niepustej wartości z wielu kolumn
- **Imputation**: Statystyczne metody wypełniania (mean, median, mode)

**Zastosowanie praktyczne:**
- Uzupełnianie brakujących wartości sensownymi defaultami
- Usuwanie rekordów z krytycznymi brakującymi danymi
- Fallback do alternatywnych źródeł danych

### Przykład 2.1: Wypełnianie wartości null (fillna)

**Cel:** Uzupełnienie brakujących wartości w kolumnach różnymi strategiami

**Podejście:**
1. Identyfikacja kolumn z null values
2. Wybór odpowiedniej strategii per kolumna
3. Zastosowanie fillna() z słownikiem wartości

In [0]:
# RESOURCE: DataFrame df_customers
# VARIABLE: df_filled - DataFrame z wypełnionymi wartościami null

# Strategia wypełniania wartości null - tylko dla kolumn które rzeczywiście mogą mieć nulls
fill_values = {
    "phone": "brak telefonu",
    "city": "Unknown",
    "state": "Unknown",
    "country": "Unknown"
}

# Wypełnienie wartości null
df_filled = df_customers.fillna(fill_values)

# Weryfikacja zmian
print("=== Porównanie przed i po fillna ===")
for col_name in fill_values.keys():
    before_nulls = df_customers.filter(F.col(col_name).isNull()).count()
    after_nulls = df_filled.filter(F.col(col_name).isNull()).count()
    print(f"{col_name:15s}: {before_nulls:3d} nulls → {after_nulls:3d} nulls")

# Przykładowe rekordy po wypełnieniu
print("\n=== Przykładowe wypełnione rekordy ===")
display(df_filled.filter(
    df_customers["phone"].isNull() | 
    df_customers["city"].isNull() |
    df_customers["state"].isNull()
).limit(5))

### Przykład 2.2: Usuwanie rekordów z wartościami null (dropna)

**Cel:** Usunięcie rekordów z brakującymi wartościami w kluczowych kolumnach

In [0]:
# RESOURCE: DataFrame df_customers
# VARIABLE: df_valid - DataFrame z usuniętymi rekordami z null w kluczowych kolumnach

# Usuń rekordy bez kluczowych informacji (customer_id jest wymagane)
df_valid = df_customers.dropna(subset=["customer_id"])

# Weryfikacja zmian
print("=== Porównanie przed i po dropna ===")
print(f"Liczba rekordów PRZED: {df_customers.count()}")
print(f"Liczba rekordów PO: {df_valid.count()}")
print(f"Usunięto rekordów: {df_customers.count() - df_valid.count()}")

# Sprawdzenie, czy są jeszcze null w customer_id
null_ids = df_valid.filter(F.col("customer_id").isNull()).count()
print(f"\nNull w customer_id po dropna: {null_ids}")

# Alternatywne użycie: dropna z how='all' (usuwa tylko jeśli wszystkie kolumny są null)
df_any_data = df_customers.dropna(how='all')
print(f"\nRekordy z jakimikolwiek danymi: {df_any_data.count()}")

### Przykład 2.3: Wybór pierwszej niepustej wartości (coalesce)

**Cel:** Użycie coalesce() do fallback między alternatywnymi źródłami danych

In [0]:
# RESOURCE: DataFrame df_customers
# VARIABLE: df_with_contact - DataFrame z nową kolumną primary_contact

from pyspark.sql.functions import coalesce, lit

# Przykład: Stwórz primary_contact wybierając pierwszą niepustą wartość
df_with_contact = df_customers.withColumn(
    "primary_contact",
    coalesce(F.col("email"), F.col("phone"), lit("brak kontaktu"))
)

# Stwórz pełny adres z dostępnych pól
df_with_contact = df_with_contact.withColumn(
    "full_address",
    coalesce(
        F.concat_ws(", ", F.col("city"), F.col("state"), F.col("country")),
        F.concat_ws(", ", F.col("city"), F.col("country")),
        F.col("country"),
        lit("Address Unknown")
    )
)

# Weryfikacja
print("=== Analiza primary_contact ===")
contact_stats = df_with_contact.groupBy("primary_contact").count().orderBy(F.desc("count"))
display(contact_stats.limit(10))

# Przykłady
print("\n=== Przykładowe rekordy z primary_contact i full_address ===")
display(df_with_contact.select("customer_id", "email", "phone", "primary_contact", "city", "state", "country", "full_address").limit(10))

## Sekcja 3: Walidacja i konwersja typów danych

**Wprowadzenie teoretyczne:**

Niepoprawne typy danych są częstym problemem podczas ładowania danych z plików tekstowych (CSV, JSON). Konwersje typu muszą być wykonane bezpiecznie z obsługą błędów, aby uniknąć utraty danych lub niepoprawnych wyników analiz.

**Kluczowe pojęcia:**
- **cast()**: Konwersja typu danych (string → int, date, timestamp)
- **to_date()**: Parsowanie stringów na DateType z określeniem formatu
- **to_timestamp()**: Parsowanie stringów na TimestampType
- **try_cast()**: Bezpieczna konwersja zwracająca null przy błędzie (Spark 3.4+)

**Zastosowanie praktyczne:**
- Walidacja typów po załadowaniu CSV z inferSchema
- Parsowanie dat w niestandardowych formatach
- Konwersja typów przed złączeniami i agregacjami

### Przykład 3.1: Konwersja typów numerycznych (cast)

**Cel:** Bezpieczna konwersja stringów na typy numeryczne z walidacją

**Podejście:**
1. Czyszczenie wartości przed konwersją (usunięcie znaków specjalnych)
2. Konwersja typu używając cast()
3. Walidacja zakresu wartości

In [0]:
# RESOURCE: DataFrame df_customers
# VARIABLE: df_typed - DataFrame z poprawnie skonwertowanymi typami

# Przykład: Walidacja dat rejestracji i dodanie flag jakości
df_typed = df_customers

# Parsowanie registration_date i walidacja zakresu
df_typed = df_typed.withColumn(
    "registration_date_parsed",
    F.to_date(F.col("registration_date"), "yyyy-MM-dd")
)

# Walidacja zakresu dat (2020-2026)
df_typed = df_typed.withColumn(
    "registration_date_valid",
    (F.col("registration_date_parsed").isNotNull()) & 
    (F.col("registration_date_parsed") >= "2020-01-01") & 
    (F.col("registration_date_parsed") <= "2026-12-31")
)

# Dodaj wiek konta w dniach
df_typed = df_typed.withColumn(
    "account_age_days",
    F.datediff(F.current_date(), F.col("registration_date_parsed"))
)

# Walidacja email format
df_typed = df_typed.withColumn(
    "email_valid",
    (F.col("email").isNotNull()) & 
    F.col("email").rlike("^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\\.[a-zA-Z]{2,}$")
)

# Statystyki konwersji
total = df_typed.count()
valid_dates = df_typed.filter(F.col("registration_date_valid") == True).count()
invalid_dates = df_typed.filter(F.col("registration_date_valid") == False).count()
valid_emails = df_typed.filter(F.col("email_valid") == True).count()

print("=== Statystyki walidacji ===")
print(f"Total: {total}")
print(f"Valid registration dates: {valid_dates} ({(valid_dates/total)*100:.1f}%)")
print(f"Invalid registration dates: {invalid_dates} ({(invalid_dates/total)*100:.1f}%)")
print(f"Valid emails: {valid_emails} ({(valid_emails/total)*100:.1f}%)")

# Przykłady nieprawidłowych wartości
if invalid_dates > 0:
    print("\n=== Przykłady nieprawidłowych dat rejestracji ===")
    display(df_typed.filter(F.col("registration_date_valid") == False).select("customer_id", "registration_date", "registration_date_parsed", "registration_date_valid").limit(5))

# Schemat po konwersji
print("\n=== Schemat po konwersji ===")
df_typed.printSchema()

### Przykład 3.2: Konwersja dat (to_date)

**Cel:** Parsowanie stringów na DateType z obsługą wielu formatów

In [0]:
# RESOURCE: DataFrame df_customers
# VARIABLE: df_with_dates - DataFrame z poprawnie sparsowanymi datami

# Przykład: Konwersja kolumny registration_date (string) na date
# Strategia: Próbuj różne popularne formaty używając coalesce()

df_with_dates = df_customers.withColumn(
    "registration_date_parsed",
    coalesce(
        F.to_date(F.col("registration_date"), "yyyy-MM-dd"),     # Format: 2024-01-15
        F.to_date(F.col("registration_date"), "dd/MM/yyyy"),     # Format: 15/01/2024
        F.to_date(F.col("registration_date"), "MM-dd-yyyy"),     # Format: 01-15-2024
        F.to_date(F.col("registration_date"), "yyyy.MM.dd"),     # Format: 2024.01.15
        F.to_date(F.col("registration_date"))                    # Automatyczne wykrywanie
    )
)

# Walidacja konwersji
total = df_with_dates.count()
parsed = df_with_dates.filter(F.col("registration_date_parsed").isNotNull()).count()
failed = df_with_dates.filter(
    F.col("registration_date").isNotNull() & 
    F.col("registration_date_parsed").isNull()
).count()

print("=== Statystyki konwersji dat ===")
print(f"Total: {total}")
print(f"Poprawnie sparsowane: {parsed} ({(parsed/total)*100:.1f}%)")
print(f"Nie udało się sparsować: {failed} ({(failed/total)*100:.1f}%)")

# Przykłady konwersji
print("\n=== Przykładowe konwersje dat ===")
display(df_with_dates.select("customer_id", "registration_date", "registration_date_parsed").limit(10))

# Rekordy z błędami parsowania
if failed > 0:
    print("\n=== Rekordy z błędami parsowania ===")
    display(df_with_dates.filter(
        F.col("registration_date").isNotNull() & 
        F.col("registration_date_parsed").isNull()
    ).select("customer_id", "registration_date", "registration_date_parsed").limit(5))

### Przykład 3.3: Konwersja timestamp i obliczenia czasowe

**Cel:** Konwersja na timestamp i wykonanie obliczeń czasowych

In [0]:
# RESOURCE: DataFrame df_with_dates
# VARIABLE: df_with_timestamp - DataFrame z timestamp i obliczeniami

from pyspark.sql.functions import to_timestamp, current_timestamp, current_date, datediff

# Konwersja registration_date_parsed na timestamp (dodaje czas 00:00:00)
df_with_timestamp = df_with_dates.withColumn(
    "registration_timestamp",
    F.to_timestamp(F.col("registration_date_parsed"))
)

# Obliczenia czasowe
df_with_timestamp = df_with_timestamp \
    .withColumn("current_date", current_date()) \
    .withColumn("days_since_registration", 
        datediff(F.col("current_date"), F.col("registration_date_parsed"))
    )

# Statystyki
print("=== Statystyki czasowe ===")
df_with_timestamp.select(
    F.min("days_since_registration").alias("min_days"),
    F.max("days_since_registration").alias("max_days"),
    F.avg("days_since_registration").alias("avg_days")
).show()

# Przykłady
print("\n=== Przykładowe obliczenia czasowe ===")
display(df_with_timestamp.select(
    "customer_id",
    "registration_date",
    "registration_date_parsed",
    "registration_timestamp",
    "days_since_registration"
).orderBy(F.desc("days_since_registration")).limit(10))

## Sekcja 4: Deduplikacja danych

**Wprowadzenie teoretyczne:**

Duplikaty są powszechnym problemem jakości danych wynikającym z błędów w systemach źródłowych, wielokrotnego ładowania tych samych danych lub błędów w procesach ETL. Strategia deduplikacji zależy od kontekstu biznesowego.

**Kluczowe pojęcia:**
- **dropDuplicates()**: Usunięcie duplikatów na podstawie wszystkich lub wybranych kolumn
- **Exact duplicates**: Rekordy identyczne we wszystkich kolumnach
- **Key duplicates**: Rekordy z tym samym kluczem biznesowym
- **Deduplication strategy**: Którą wersję rekordu zachować (first, last, newest)

**Zastosowanie praktyczne:**
- Usuwanie duplikatów po key columns (customer_id)
- Zachowanie najnowszej wersji rekordu (last insert wins)
- Identyfikacja potencjalnych duplikatów (fuzzy matching)

### Przykład 4.1: Deduplikacja - wszystkie kolumny

**Cel:** Usunięcie rekordów całkowicie identycznych (exact duplicates)

In [0]:
# RESOURCE: DataFrame df_customers
# VARIABLE: df_distinct - DataFrame bez dokładnych duplikatów

# Usunięcie exact duplicates (wszystkie kolumny identyczne)
df_distinct = df_customers.distinct()

# Statystyki
total = df_customers.count()
distinct = df_distinct.count()
duplicates = total - distinct

print("=== Deduplikacja - wszystkie kolumny ===")
print(f"Total rekordów: {total}")
print(f"Unikalne rekordy: {distinct}")
print(f"Usunięte duplikaty: {duplicates}")
print(f"Duplication rate: {(duplicates/total)*100:.1f}%")

# Identyfikacja duplikatów przed usunięciem
from pyspark.sql.functions import count as spark_count

duplicated_records = df_customers \
    .groupBy(df_customers.columns) \
    .agg(spark_count("*").alias("count")) \
    .filter(F.col("count") > 1) \
    .orderBy(F.desc("count"))

if duplicated_records.count() > 0:
    print("\n=== Przykłady zduplikowanych rekordów ===")
    display(duplicated_records.limit(5))

### Przykład 4.2: Deduplikacja per key columns

**Cel:** Usunięcie duplikatów na podstawie klucza biznesowego (customer_id), zachowując najnowszy rekord

In [0]:
# RESOURCE: DataFrame df_customers
# VARIABLE: df_deduped - DataFrame z usuniętymi duplikatami per customer_id

# Strategia 1: dropDuplicates() - zachowuje pierwszy napotkany rekord
df_deduped_simple = df_customers.dropDuplicates(["customer_id"])

print("=== Deduplikacja per customer_id (strategia simple) ===")
print(f"Przed: {df_customers.count()} rekordów")
print(f"Po: {df_deduped_simple.count()} rekordów")
print(f"Usunięto: {df_customers.count() - df_deduped_simple.count()} duplikatów")

# Strategia 2: Window function - zachowaj najnowszy rekord (jeśli mamy timestamp)
# Zakładamy, że mamy kolumnę created_at lub inny timestamp

if "created_at" in df_customers.columns or "last_updated" in df_customers.columns:
    from pyspark.sql.window import Window
    
    timestamp_col = "created_at" if "created_at" in df_customers.columns else "last_updated"
    
    # Okno partycjonowane po customer_id, sortowane po timestamp desc
    window_spec = Window.partitionBy("customer_id").orderBy(F.desc(timestamp_col))
    
    df_deduped = df_customers \
        .withColumn("row_num", F.row_number().over(window_spec)) \
        .filter(F.col("row_num") == 1) \
        .drop("row_num")
    
    print(f"\n=== Deduplikacja per customer_id (strategia: najnowszy rekord) ===")
    print(f"Przed: {df_customers.count()} rekordów")
    print(f"Po: {df_deduped.count()} rekordów")
    print(f"Usunięto: {df_customers.count() - df_deduped.count()} duplikatów")
else:
    # Jeśli brak timestamp, użyj prostej strategii
    df_deduped = df_deduped_simple
    print("\n(Brak kolumny timestamp - użyto prostej strategii)")

# Identyfikacja duplikatów przed usunięciem
duplicate_ids = df_customers \
    .groupBy("customer_id") \
    .agg(spark_count("*").alias("count")) \
    .filter(F.col("count") > 1) \
    .orderBy(F.desc("count"))

if duplicate_ids.count() > 0:
    print(f"\n=== {duplicate_ids.count()} customer_id z duplikatami ===")
    display(duplicate_ids.limit(10))
    
    # Przykłady duplikatów
    sample_duplicate_id = duplicate_ids.first()["customer_id"]
    print(f"\n=== Przykład duplikatów dla customer_id={sample_duplicate_id} ===")
    display(df_customers.filter(F.col("customer_id") == sample_duplicate_id))

## Sekcja 5: Standardyzacja danych

**Wprowadzenie teoretyczne:**

Standaryzacja polega na ujednoliceniu formatów i reprezentacji danych zgodnie z ustalonymi regułami biznesowymi. Niestandardowe dane (różne case, whitespace, formaty) utrudniają analizy, złączenia i agregacje.

**Kluczowe pojęcia:**
- **Text standardization**: trim(), lower(), upper(), initcap()
- **Pattern standardization**: regexp_replace() dla kodów, telefonów
- **Format standardization**: Ujednolicenie formatów dat, adresów
- **Categorical standardization**: Mapowanie wariantów na standardowe wartości

**Zastosowanie praktyczne:**
- Ujednolicenie wielkości liter w polach tekstowych
- Usunięcie whitespace z początku i końca
- Standaryzacja kodów krajów, telefonów, kodów pocztowych
- Konsolidacja wariantów kategorii (Active/active/ACTIVE → Active)

### Przykład 5.1: Standaryzacja tekstu

**Cel:** Oczyszczenie i standaryzacja pól tekstowych (trim, case, whitespace)

In [0]:
# RESOURCE: DataFrame df_customers
# VARIABLE: df_standardized - DataFrame ze standaryzowanymi polami tekstowymi

# Standaryzacja pól tekstowych
df_standardized = df_customers

# 1. Trim whitespace z wszystkich string columns
for col_name in ["first_name", "last_name", "email", "phone", "city", "state", "country"]:
    if col_name in df_standardized.columns:
        df_standardized = df_standardized.withColumn(
            col_name,
            F.trim(F.col(col_name))
        )

# 2. Standaryzacja konkretnych kolumn
# first_name, last_name: Title Case
df_standardized = df_standardized.withColumn(
    "first_name",
    F.initcap(F.col("first_name"))
).withColumn(
    "last_name", 
    F.initcap(F.col("last_name"))
)

# email: lowercase (standard dla emaili)
df_standardized = df_standardized.withColumn(
    "email",
    F.lower(F.col("email"))
)

# country: uppercase (standard ISO dla kodów krajów)
df_standardized = df_standardized.withColumn(
    "country",
    F.upper(F.col("country"))
)

# city: Title Case
df_standardized = df_standardized.withColumn(
    "city",
    F.initcap(F.col("city"))
)

# Porównanie przed i po
print("=== Porównanie standaryzacji ===")
display(df_customers.select("first_name", "last_name", "email", "city", "country").limit(5))
print("\n↓↓↓ PO STANDARYZACJI ↓↓↓\n")
display(df_standardized.select("first_name", "last_name", "email", "city", "country").limit(5))

### Przykład 5.2: Standaryzacja kodów i kategorii

**Cel:** Ujednolicenie formatów kodów i mapowanie wariantów kategorii

In [0]:
# RESOURCE: DataFrame df_standardized
# VARIABLE: df_codes_standardized - DataFrame ze standaryzowanymi kodami

df_codes_standardized = df_standardized

# 1. Standaryzacja phone numbers (usunięcie wszystkich non-digit, format międzynarodowy)
if "phone" in df_codes_standardized.columns:
    df_codes_standardized = df_codes_standardized.withColumn(
        "phone_standardized",
        F.when(F.col("phone").isNotNull(),
            F.concat(
                F.when(F.col("phone").startswith("+"), "")
                 .otherwise("+1-"),  # Domyślny prefiks dla USA
                F.regexp_replace(F.col("phone"), "[^0-9]", "")
            )
        ).otherwise(F.col("phone"))
    )

# 2. Standaryzacja customer_segment (konsystentne nazewnictwo)
if "customer_segment" in df_codes_standardized.columns:
    df_codes_standardized = df_codes_standardized.withColumn(
        "customer_segment_standardized",
        F.when(F.upper(F.trim(F.col("customer_segment"))) == "PREMIUM", "Premium")
         .when(F.upper(F.trim(F.col("customer_segment"))) == "STANDARD", "Standard")
         .when(F.upper(F.trim(F.col("customer_segment"))) == "BASIC", "Basic")
         .otherwise("Unknown")
    )

# 3. Standaryzacja country codes (3-letter ISO codes jako przykład)
df_codes_standardized = df_codes_standardized.withColumn(
    "country_iso",
    F.when(F.upper(F.col("country")) == "USA", "USA")
     .when(F.upper(F.col("country")) == "POLAND", "POL")
     .when(F.upper(F.col("country")) == "GERMANY", "DEU")
     .when(F.upper(F.col("country")) == "FRANCE", "FRA")
     .otherwise(F.upper(F.col("country")))
)

# Weryfikacja standaryzacji
print("=== Standaryzacja kodów i kategorii ===")

if "phone" in df_codes_standardized.columns:
    print("\n--- Phone numbers ---")
    display(df_codes_standardized.select("phone", "phone_standardized").limit(5))

if "customer_segment" in df_codes_standardized.columns:
    print("\n--- Customer segments ---")
    display(df_codes_standardized.groupBy("customer_segment", "customer_segment_standardized").count().orderBy("customer_segment"))

print("\n--- Country codes ---")
display(df_codes_standardized.groupBy("country", "country_iso").count().orderBy("country").limit(10))

## Porównanie PySpark vs SQL

**DataFrame API (PySpark):**

In [0]:
# Przykład: Kompleksowe czyszczenie danych w PySpark

df_cleaned_pyspark = df_customers \
    .fillna({"city": "Unknown", "country": "USA"}) \
    .dropna(subset=["customer_id"]) \
    .withColumn("first_name", F.trim(F.initcap(F.col("first_name")))) \
    .withColumn("last_name", F.trim(F.initcap(F.col("last_name")))) \
    .withColumn("email", F.lower(F.trim(F.col("email")))) \
    .dropDuplicates(["customer_id"])

print(f"Rekordy po czyszczeniu (PySpark): {df_cleaned_pyspark.count()}")
display(df_cleaned_pyspark.limit(5))

**SQL Equivalent:**

```sql
# Najpierw utwórz temporary view
df_customers.createOrReplaceTempView("customers_raw")

# SQL approach
df_cleaned_sql = spark.sql("""
SELECT DISTINCT
    customer_id,
    TRIM(INITCAP(first_name)) as first_name,
    TRIM(INITCAP(last_name)) as last_name,
    LOWER(TRIM(email)) as email,
    COALESCE(city, 'Unknown') as city,
    COALESCE(country, 'Unknown') as country,
    phone,
    registration_date,
    customer_segment
FROM customers_raw
WHERE customer_id IS NOT NULL
    AND email IS NOT NULL
    AND email LIKE '%@%'
""")

print(f"Rekordy po czyszczeniu (SQL): {df_cleaned_sql.count()}")
display(df_cleaned_sql.limit(5))
```

In [0]:
# Najpierw utwórz temporary view
df_customers.createOrReplaceTempView("customers_raw")

# SQL approach
df_cleaned_sql = spark.sql("""
SELECT DISTINCT
    customer_id,
    TRIM(INITCAP(first_name)) as first_name,
    TRIM(INITCAP(last_name)) as last_name,
    LOWER(TRIM(email)) as email,
    COALESCE(city, 'Unknown') as city,
    COALESCE(country, 'USA') as country,
    phone,
    registration_date,
    customer_segment
FROM customers_raw
WHERE customer_id IS NOT NULL
""")

print(f"Rekordy po czyszczeniu (SQL): {df_cleaned_sql.count()}")
display(df_cleaned_sql.limit(5))

**Porównanie:**
- **Wydajność**: Identyczna - Catalyst optimizer kompiluje oba do tego samego execution plan
- **Kiedy używać PySpark**: Łańcuchowe transformacje, dynamiczne pipeline'y, integracja z Python libraries
- **Kiedy używać SQL**: Złożone joiny, window functions, analitycy preferujący SQL syntax
- **Best practice**: Używaj tego, co jest bardziej czytelne dla zespołu i przypadku użycia

## Walidacja i weryfikacja

### Checklist - Co powinieneś uzyskać:
- [ ] Dane załadowane z dataset/ (customers.csv używając DATASET_BASE_PATH)
- [ ] Data profiling report wykonany
- [ ] Null values obsłużone (fillna/dropna/coalesce)
- [ ] Typy danych skonwertowane i zwalidowane (cast, to_date)
- [ ] Duplikaty usunięte (dropDuplicates)
- [ ] Pola tekstowe standaryzowane (trim, case)
- [ ] Kody i kategorie zunifikowane
- [ ] Brak błędów w execution

### Komendy weryfikacyjne:

In [0]:
# Weryfikacja wyników - użyj df_codes_standardized jako final

df_final = df_codes_standardized

print("=== WERYFIKACJA WYNIKÓW ===\n")

# 1. Podstawowe statystyki
print("1. Podstawowe statystyki:")
print(f"   Liczba rekordów: {df_final.count()}")
print(f"   Liczba kolumn: {len(df_final.columns)}")

# 2. Null values per kolumna (sprawdź tylko pierwsze 10 kolumn)
print("\n2. Null values per kolumna (próbka):")
for col_name in df_final.columns[:10]:  # Pokaż tylko pierwsze 10
    null_count = df_final.filter(F.col(col_name).isNull()).count()
    print(f"   {col_name:20s}: {null_count:4d} nulls")

# 3. Duplikaty
print("\n3. Analiza duplikatów:")
total = df_final.count()
distinct = df_final.distinct().count()
print(f"   Total: {total}, Distinct: {distinct}, Duplicates: {total - distinct}")

# 4. Schemat danych (tylko typy)
print("\n4. Schemat danych:")
for field in df_final.schema.fields[:10]:  # Pokaż tylko pierwsze 10
    print(f"   {field.name:20s}: {field.dataType}")

# 5. Podgląd danych
print("\n5. Podgląd wyczyszczonych danych:")
display(df_final.select("customer_id", "first_name", "last_name", "email", "customer_segment", "country").limit(10))

# 6. Testy asercyjne
try:
    assert df_final.count() > 0, "BŁĄD: DataFrame jest pusty"
    assert df_final.filter(F.col("customer_id").isNull()).count() == 0, "BŁĄD: Null values w customer_id"
    assert df_final.count() == df_final.dropDuplicates(["customer_id"]).count(), "BŁĄD: Duplikaty w customer_id"
    print("\n✓ Wszystkie testy przeszły pomyślnie!")
except AssertionError as e:
    print(f"\n✗ Test failed: {e}")

## Troubleshooting

### Problem 1: Błąd przy wczytywaniu z Volume
**Objawy:**
- FileNotFoundException
- "Path does not exist"

**Rozwiązanie:**
```python
# Sprawdź zawartość DATASET_BASE_PATH
import os
if os.path.exists(DATASET_BASE_PATH):
    print(f"✓ DATASET_BASE_PATH exists: {DATASET_BASE_PATH}")
    for folder in os.listdir(DATASET_BASE_PATH):
        print(f"  - {folder}/")
else:
    print(f"✗ DATASET_BASE_PATH not found: {DATASET_BASE_PATH}")

# Lista plików w customers/
customers_dir = f"{DATASET_BASE_PATH}/customers"
if os.path.exists(customers_dir):
    for file in os.listdir(customers_dir):
        print(f"  - {file}")
```

### Problem 2: InferSchema daje niepoprawne typy
**Objawy:** 
- Kolumny numeryczne jako StringType
- Daty nie są rozpoznawane

**Rozwiązanie:**
Zdefiniuj schemat jawnie:
```python
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

schema = StructType([
    StructField("customer_id", StringType(), False),
    StructField("first_name", StringType(), True),
    StructField("last_name", StringType(), True),
    StructField("email", StringType(), True),
    StructField("phone", StringType(), True),
    StructField("city", StringType(), True),
    StructField("state", StringType(), True),
    StructField("country", StringType(), True),
    StructField("registration_date", StringType(), True),
    StructField("customer_segment", StringType(), True)
])

df = spark.read.format("csv").option("header", "true").schema(schema).load(customers_path)
```

### Problem 3: Konwersja dat zwraca null
**Objawy:**
- to_date() zwraca null dla wszystkich wartości
- Niepoprawny format daty

**Rozwiązanie:**
```python
# Sprawdź przykładowe wartości
df.select("registration_date").show(5, truncate=False)

# Wypróbuj różne formaty
df.select(
    F.col("registration_date"),
    F.to_date(F.col("registration_date"), "yyyy-MM-dd").alias("format1"),
    F.to_date(F.col("registration_date"), "dd/MM/yyyy").alias("format2"),
    F.to_date(F.col("registration_date"), "MM-dd-yyyy").alias("format3")
).show(5)
```

### Debugging tips:
- Użyj `.explain()` aby zobaczyć plan wykonania
- Sprawdź typy: `df.printSchema()`
- Podgląd danych: `display(df.limit(10))`
- Monitoruj null counts po każdej transformacji
- Użyj `.show(truncate=False)` dla pełnych wartości

## Best Practices

### Wydajność:
- Użyj jawnych schematów zamiast inferSchema w produkcji (szybsze, bardziej przewidywalne)
- dropDuplicates() jest operacją shuffle - użyj ostrożnie na dużych danych
- Partycjonuj dane przed deduplikacją per klucz biznesowy
- Cache() DataFrames używanych wielokrotnie

### Jakość kodu:
- Zawsze waliduj dane po konwersji typów (null counts, value ranges)
- Dodawaj flagi jakości (email_valid, date_parsed_success) dla auditability
- Loguj statystyki czyszczenia (ile rekordów usunięto, wypełniono)
- Zachowaj oryginalne wartości w osobnych kolumnach dla troubleshootingu

### Data Quality:
- Definiuj jasne reguły biznesowe dla każdego pola (allowed values, ranges)
- Implementuj quality checks jako assertions w pipeline
- Używaj coalesce() dla fallback strategies zamiast prostego fillna()
- Dokumentuj decyzje o czyszczeniu (dlaczego usunięto, dlaczego fillna)

### Governance:
- Wszystkie transformacje muszą być deterministyczne (powtarzalne)
- Audit trail: timestamp, user, operation w metadata columns
- Zachowaj raw data w Bronze layer (nigdy nie nadpisuj źródła)
- Używaj DATASET_BASE_PATH z 00_setup.ipynb dla spójnych ścieżek do danych

## Podsumowanie

### Co zostało osiągnięte:
- Załadowanie danych z folderu dataset/ (używając DATASET_BASE_PATH)
- Data profiling i identyfikacja problemów jakości
- Obsługa wartości null (fillna, dropna, coalesce)
- Walidacja i konwersja typów (cast, to_date, to_timestamp)
- Deduplikacja rekordów (distinct, dropDuplicates, window functions)
- Standaryzacja tekstu i kodów (trim, case, regexp_replace)
- Porównanie PySpark vs SQL approaches

### Kluczowe wnioski:
1. **Data profiling first**: Zawsze analizuj dane przed rozpoczęciem czyszczenia
2. **Context matters**: Strategia czyszczenia zależy od kontekstu biznesowego
3. **Validation is critical**: Zawsze waliduj wyniki konwersji i transformacji
4. **Document decisions**: Loguj statystyki i decyzje dla auditability

### Quick Reference - Najważniejsze komendy:

| Operacja | PySpark | SQL |
|----------|---------|-----|
| Fill nulls | `df.fillna({"col": "value"})` | `COALESCE(col, 'value')` |
| Drop nulls | `df.dropna(subset=["col"])` | `WHERE col IS NOT NULL` |
| Parse date | `to_date(col("date"), "yyyy-MM-dd")` | `TO_DATE(date, 'yyyy-MM-dd')` |
| Remove duplicates | `df.dropDuplicates(["id"])` | `SELECT DISTINCT` |
| Trim whitespace | `trim(col("name"))` | `TRIM(name)` |
| Lowercase | `lower(col("email"))` | `LOWER(email)` |
| Uppercase | `upper(col("country"))` | `UPPER(country)` |
| Title case | `initcap(col("city"))` | `INITCAP(city)` |

### Następne kroki:
- **Kolejny notebook**: 05_views_workflows.ipynb - Persistent views i basic workflows
- **Warsztat praktyczny**: 02_transformations_cleaning_workshop.ipynb
- **Materiały dodatkowe**: 
  - Databricks Data Quality Guide
  - Delta Lake best practices documentation
- **Zadanie domowe**: Zastosuj poznane techniki czyszczenia na dataset orders (orders_batch.json z dataset/)

## Czyszczenie zasobów

Posprzątaj zasoby utworzone podczas notebooka:

In [0]:
# Opcjonalne czyszczenie zasobów testowych
# UWAGA: Uruchom tylko jeśli chcesz usunąć wszystkie utworzone dane

# Usuń temporary views
spark.catalog.dropTempView("customers_raw")

# Wyczyść cache
spark.catalog.clearCache()

print("Temporary views i cache zostały wyczyszczone")
print("Dane źródłowe w Volume pozostają nienaruszone")